In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from fancyimpute import KNN    
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, auc, roc_curve, precision_recall_curve, average_precision_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
weekly = pd.read_csv(filepath_or_buffer='Courier_weekly_data.csv')

In [4]:
weekly.head(20)

,courier,week,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17
0,3767,2,6,34,38,0.0789,0.9211,140.4737,0.1316,2162.4737,0.7632,7.340776,8,20.208158,5.236316,0.8158,43.384804,1,19
1,3767,4,-1,42,37,0.0000,1.0000,135.5946,0.0811,2097.4054,0.9459,11.883784,19,18.855405,5.689459,0.8919,35.078042,3,11
2,3767,5,24,41,43,0.0233,0.9767,131.0930,0.0233,2043.8837,0.9302,7.072100,16,18.925116,5.138605,0.9302,31.455285,1,10
3,3767,6,-22,65,66,0.0606,0.9394,120.1515,0.0000,2124.2727,0.7727,7.356567,33,18.259697,4.704394,0.7879,34.252991,1,30
4,6282,2,9,33,27,0.0741,0.9259,100.0000,0.0370,4075.7407,0.8889,8.501233,5,26.863704,4.828519,0.8889,46.478114,1,4
5,6282,3,-20,42,56,0.0536,0.9464,113.4821,0.0357,4777.0714,0.9107,8.210125,16,23.651786,5.553571,0.9107,79.407407,2,5
6,6282,4,9,22,32,0.1250,0.8750,105.0000,0.0938,5744.1875,0.8125,8.285422,12,18.180937,5.834375,0.7813,87.250000,1,2
7,6282,5,21,31,48,0.0417,0.9583,117.2500,0.0833,4011.7708,0.9167,9.768052,17,20.346667,5.615417,0.9167,85.083333,2,9
8,6282,6,-12,52,72,0.0694,0.9306,107.6389,0.0417,4000.3333,0.8472,7.736114,17,21.941111,5.093056,0.8750,73.904915,1,13
9,6282,7,1,40,56,0.0893,0.9107,124.3750,0.0893,3192.0000,0.9107,7.405355,11,18.591071,5.353214,0.9107,65.618750,2,4


In [5]:
weekly[(weekly.week==11) | (weekly.week==10) | (weekly.week==9)]['courier'].drop_duplicates().count()


387

In [7]:
weeks=weekly.copy()

In [8]:
def week_label(row):
    courier_set=weekly[(weekly.courier==row['courier']) & ((weekly.week==9) | (weekly.week==10) |(weekly.week==11))]
    if courier_set['courier'].count() == 0:
        label=1
    else: 
        label=0
    return label

In [10]:
weeks['label']=weeks.apply(week_label, axis=1)


In [11]:
weeks=weeks[(weeks.week<8)]

In [19]:
weeks.head(20)

,courier,week,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,label
0,3767,2,6,34,38,0.0789,0.9211,140.4737,0.1316,2162.4737,0.7632,7.340776,8,20.208158,5.236316,0.8158,43.384804,1,19,1
1,3767,4,-1,42,37,0.0000,1.0000,135.5946,0.0811,2097.4054,0.9459,11.883784,19,18.855405,5.689459,0.8919,35.078042,3,11,1
2,3767,5,24,41,43,0.0233,0.9767,131.0930,0.0233,2043.8837,0.9302,7.072100,16,18.925116,5.138605,0.9302,31.455285,1,10,1
3,3767,6,-22,65,66,0.0606,0.9394,120.1515,0.0000,2124.2727,0.7727,7.356567,33,18.259697,4.704394,0.7879,34.252991,1,30,1
4,6282,2,9,33,27,0.0741,0.9259,100.0000,0.0370,4075.7407,0.8889,8.501233,5,26.863704,4.828519,0.8889,46.478114,1,4,0
5,6282,3,-20,42,56,0.0536,0.9464,113.4821,0.0357,4777.0714,0.9107,8.210125,16,23.651786,5.553571,0.9107,79.407407,2,5,0
6,6282,4,9,22,32,0.1250,0.8750,105.0000,0.0938,5744.1875,0.8125,8.285422,12,18.180937,5.834375,0.7813,87.250000,1,2,0
7,6282,5,21,31,48,0.0417,0.9583,117.2500,0.0833,4011.7708,0.9167,9.768052,17,20.346667,5.615417,0.9167,85.083333,2,9,0
8,6282,6,-12,52,72,0.0694,0.9306,107.6389,0.0417,4000.3333,0.8472,7.736114,17,21.941111,5.093056,0.8750,73.904915,1,13,0
9,6282,7,1,40,56,0.0893,0.9107,124.3750,0.0893,3192.0000,0.9107,7.405355,11,18.591071,5.353214,0.9107,65.618750,2,4,0


In [13]:
def max_consecutive(vector):
    longest = 0
    current = 0
    for num in vector:
        if num == 1:
            current += 1
        else:
            longest = max(longest, current)
            current = 0

    return max(longest, current)
    

In [22]:
max_consecutive([0,1,1,1,0,0,1,0,1,1,1,1,1,1,0])

6

In [29]:
def max_streak(series):
    week_vector=[0,0,0,0,0,0,0,0]
    
    for i in series:
        week_vector[i]=1
    streak=max_consecutive(week_vector)
    return streak
    
        
        

In [33]:
def lifetime_mean(series):
    return np.sum(series)/8

In [39]:
gold=weeks.groupby(['courier','label'], as_index=False ).agg({'week':['count', max_streak],'feature_1':['mean',lifetime_mean],'feature_2':['mean',lifetime_mean],'feature_3':['mean',lifetime_mean],'feature_4':['mean',lifetime_mean],'feature_5':['mean',lifetime_mean],'feature_6':['mean',lifetime_mean],'feature_7':['mean',lifetime_mean],'feature_8':['mean',lifetime_mean]})

In [40]:
weeks[(weeks.courier==101552)]

,courier,week,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,label
151,101552,0,17,85,96,0.0625,0.9375,115.4167,0.1146,2526.3125,0.8646,9.938890,38,19.448333,4.831042,0.8542,47.183007,5,12,0
152,101552,1,-12,102,98,0.0306,0.9694,122.8469,0.1735,2602.7653,0.8265,10.782992,41,19.302449,5.267857,0.8673,44.454521,4,23,0
153,101552,2,5,90,95,0.0737,0.9263,110.0632,0.2000,2614.3158,0.8632,10.320528,37,23.150000,4.762842,0.8842,48.739506,4,21,0
154,101552,3,-9,95,88,0.0341,0.9659,113.5682,0.1818,2583.0455,0.9205,9.580684,33,22.335909,4.818295,0.8977,39.243567,1,19,0
155,101552,4,14,86,75,0.0000,1.0000,121.9467,0.2667,2316.0000,0.9333,9.942443,30,21.335467,4.931467,1.0000,37.920543,2,6,0
156,101552,5,-13,100,107,0.0654,0.9346,118.7477,0.0374,2363.8318,0.8318,8.228350,38,21.186168,4.572243,0.8318,43.220833,5,17,0
157,101552,6,13,87,73,0.0411,0.9589,122.3973,0.0137,2294.2329,0.8767,8.809360,51,23.320411,4.752466,0.8356,31.955300,2,10,0
158,101552,7,-6,100,80,0.0375,0.9625,110.1375,0.0500,2481.0875,0.8500,8.936045,52,19.992125,4.389875,0.8625,32.114722,1,17,0


In [41]:
gold

courier label  week             feature_1                 feature_2  \
                  count max_streak       mean lifetime_mean        mean   
0      3767     1     4          3   1.750000         0.875   45.500000   
1      6282     0     6          6   1.333333         1.000   36.666667   
2     10622     1     2          2  -3.500000        -0.875   84.500000   
3     13096     0     2          2   0.000000         0.000   59.000000   
4     14261     1     8          8  -3.625000        -3.625   75.500000   
5     18869     1     2          1 -17.500000        -4.375  117.500000   
6     18920     1     5          5 -19.800000       -12.375  122.600000   
7     19450     0     1          1  52.000000         6.500   89.000000   
8     39517     0     5          3   7.600000         4.750   61.000000   
9     39519     0     5          3   9.000000         5.625   69.000000   
10    41143     0     6          3  11.333333         8.500   98.500000   
11    41236     1     3          2   3.333333         1.250   82.333333   
12    41486     0     7          6  -2.285714        -2.000  119.142857   
13    41488     0     6          4   5.000000         3.750   49.833333   
14    43746     0     6          4  -4.666667        -3.500  112.500000   
15    50907     0     3          2 -26.000000        -9.750   83.333333   
16    54968     1     1          1  15.000000         1.875   44.000000   
17    68469     0     1          1   7.000000         0.875  101.000000   
18    69348     0     5          4  -3.400000        -2.125   66.200000   
19    75053     1     6          5  -2.666667        -2.000  135.333333   
20    82968     1     6          4  -0.666667        -0.500  104.333333   
21    86596     0     6          5  -1.166667        -0.875  103.833333   
22    87117     0     4          2 -20.000000       -10.000   94.250000   
23    87766     0     5          5  -5.600000        -3.500   80.200000   
24    94233     1     2          2  -8.500000        -2.125   14.500000   
25   101552     0     8          8   1.125000         1.125   93.125000   
26   103832     0     7          6  -1.142857        -1.000  102.142857   
27   107223     0     7          6   0.142857         0.125   42.428571   
28   107231     1     5          2   4.000000         2.500   87.800000   
29   107556     0     1          1 -10.000000        -1.250   28.000000   
..      ...   ...   ...        ...        ...           ...         ...   
699  515676     1     1          1 -11.000000        -1.375   11.000000   
700  515681     1     1          1   7.000000         0.875    6.000000   
701  515698     1     3          3  -2.666667        -1.000   13.000000   
702  515707     1     2          1 -38.000000        -9.500   38.000000   
703  516944     0     4          4   0.250000         0.125   22.000000   
704  516968     0     4          3   8.000000         4.000   41.750000   
705  516974     0     6          3  -6.166667        -4.625   36.166667   
706  516985     1     2          1 -35.000000        -8.750   39.000000   
707  517018     0     5          2  -3.000000        -1.875   18.600000   
708  517029     0     4          3  -7.750000        -3.875   34.750000   
709  517033     1     1          1   0.000000         0.000   11.000000   
710  517044     1     6          6  -0.166667        -0.125   36.666667   
711  517058     1     6          6   2.000000         1.500   41.833333   
712  517073     0     8          8   1.125000         1.125   48.500000   
713  517094     0     2          1  -5.000000        -1.250    9.500000   
714  517449     0     6          5   6.333333         4.750   43.166667   
715  517458     1     4          2   1.000000         0.500   25.750000   
716  517465     0     5          2   0.800000         0.500   21.200000   
717  517476     0     8          8   2.250000         2.250   46.625000   
718  517495     1     2          2 -10.500000        -2.625   20.500000   
719  517510     1     5      